Trabalho de Inteligência Artificial - Algoritmos Busca</br>
Implementação de algoritmos para resolver o Cubo de Rubik</br>
Autor: Carlos Matheus R Martins</br>
Matrícula: 514690

In [1]:

import numpy as np
import time
import random
from collections import deque
from heapq import heappush, heappop

In [2]:
class RubiksCube:
    """Representação do Cubo de Rubik usando arrays NumPy"""

    def __init__(self, tamanho=3):
        self.tamanho = tamanho
        self.estado = np.array([
            np.full((tamanho, tamanho), face_id, dtype=np.int8)
            for face_id in range(6)
        ])

    def criar_copia(self):
        """Cria uma cópia independente do cubo"""
        novo_cubo = RubiksCube(self.tamanho)
        novo_cubo.estado = np.copy(self.estado)
        return novo_cubo

    def solver(self):
        """Verifica se todas as faces estão uniformes"""
        for face_id in range(6):
            if not np.all(self.estado[face_id] == face_id):
                return False
        return True

    def girar_face_horario(self, face_idx):
        """Rotaciona uma face 90 graus no sentido horário"""
        self.estado[face_idx] = np.rot90(self.estado[face_idx], k=-1)

    def movimento_frente(self):
        """Executa movimento F (Face frontal horário)"""
        n = self.tamanho
        self.girar_face_horario(0)

        temp = np.copy(self.estado[4, n-1, :])
        self.estado[4, n-1, :] = np.flip(self.estado[3, :, n-1])
        self.estado[3, :, n-1] = self.estado[5, 0, :]
        self.estado[5, 0, :] = np.flip(self.estado[1, :, 0])
        self.estado[1, :, 0] = np.flip(temp)

    def movimento_direita(self):
        """Executa movimento R (Face direita horário)"""
        n = self.tamanho
        self.girar_face_horario(1)

        temp = np.copy(self.estado[4, :, n-1])
        self.estado[4, :, n-1] = self.estado[0, :, n-1]
        self.estado[0, :, n-1] = self.estado[5, :, n-1]
        self.estado[5, :, n-1] = np.flip(self.estado[2, :, 0])
        self.estado[2, :, 0] = np.flip(temp)

    def movimento_cima(self):
        """Executa movimento U (Face superior horário)"""
        n = self.tamanho
        self.girar_face_horario(4)

        temp = np.copy(self.estado[0, 0, :])
        self.estado[0, 0, :] = self.estado[1, 0, :]
        self.estado[1, 0, :] = self.estado[2, 0, :]
        self.estado[2, 0, :] = self.estado[3, 0, :]
        self.estado[3, 0, :] = temp

    def movimento_tras(self):
        """Executa movimento B (Face traseira horário)"""
        n = self.tamanho
        self.girar_face_horario(2)

        temp = np.copy(self.estado[4, 0, :])
        self.estado[4, 0, :] = self.estado[1, :, n-1]
        self.estado[1, :, n-1] = np.flip(self.estado[5, n-1, :])
        self.estado[5, n-1, :] = self.estado[3, :, 0]
        self.estado[3, :, 0] = temp

    def movimento_esquerda(self):
        """Executa movimento L (Face esquerda horário)"""
        n = self.tamanho
        self.girar_face_horario(3)

        temp = np.copy(self.estado[4, :, 0])
        self.estado[4, :, 0] = np.flip(self.estado[2, :, n-1])
        self.estado[2, :, n-1] = np.flip(self.estado[5, :, 0])
        self.estado[5, :, 0] = self.estado[0, :, 0]
        self.estado[0, :, 0] = temp

    def movimento_baixo(self):
        """Executa movimento D (Face inferior horário)"""
        n = self.tamanho
        self.girar_face_horario(5)

        temp = np.copy(self.estado[0, n-1, :])
        self.estado[0, n-1, :] = self.estado[3, n-1, :]
        self.estado[3, n-1, :] = self.estado[2, n-1, :]
        self.estado[2, n-1, :] = self.estado[1, n-1, :]
        self.estado[1, n-1, :] = temp

    def executar_movimento(self, comando):
        """Executa um movimento baseado no comando"""
        movimentos_basicos = {
            'F': self.movimento_frente,
            'R': self.movimento_direita,
            'U': self.movimento_cima,
            'B': self.movimento_tras,
            'L': self.movimento_esquerda,
            'D': self.movimento_baixo
        }

        if comando in movimentos_basicos:
            movimentos_basicos[comando]()
        elif comando.endswith("'"):
            base = comando[:-1]
            if base in movimentos_basicos:
                for _ in range(3):
                    movimentos_basicos[base]()

    def obter_movimentos_possiveis(self):
        """Retorna lista de todos os movimentos possíveis"""
        return ['F', 'R', 'U', 'B', 'L', 'D', "F'", "R'", "U'", "B'", "L'", "D'"]

    def embaralhar_cubo(self, quantidade_movimentos):
        """Embaralha o cubo com movimentos aleatórios"""
        sequencia = []
        for _ in range(quantidade_movimentos):
            movimento = random.choice(self.obter_movimentos_possiveis())
            self.executar_movimento(movimento)
            sequencia.append(movimento)
        return sequencia

    def __hash__(self):
        return hash(self.estado.tobytes())

    def __eq__(self, outro):
        return np.array_equal(self.estado, outro.estado)

In [3]:
class EstatisticasBusca:
    """Classe para armazenar resultados dos algoritmos de busca"""

    def __init__(self):
        self.caminho_solucao = None
        self.tempo_execucao = 0
        self.estados_explorados = 0
        self.memoria_usada = 0
        self.sucesso = False

In [4]:
# HEURÍSTICA

def heuristica_padrao_cruzado(cubo):
    """Heurística que verifica padrões em cruz nas faces"""
    pontuacao = 0
    n = cubo.tamanho
    centro = n // 2

    for face_id in range(6):
        face = cubo.estado[face_id]

        if face[centro, centro] != face_id:
            pontuacao += 3

        if n > 1:
            bordas_corretas = 0
            if face[0, centro] == face_id: bordas_corretas += 1
            if face[n-1, centro] == face_id: bordas_corretas += 1
            if face[centro, 0] == face_id: bordas_corretas += 1
            if face[centro, n-1] == face_id: bordas_corretas += 1

            pontuacao += (4 - bordas_corretas) * 2

    return pontuacao

In [5]:
# BFS
def busca_em_largura(cubo_inicial, profundidade_max=50):
    """Implementação da Busca em Largura (BFS)"""
    resultado = EstatisticasBusca()
    inicio = time.time()

    if cubo_inicial.solver():
        resultado.tempo_execucao = time.time() - inicio
        resultado.caminho_solucao = []
        resultado.sucesso = True
        return resultado

    fila = deque([(cubo_inicial, [])])
    estados_visitados = {cubo_inicial}

    while fila:
        # Calcula pico de memória (fila + visitados)
        memoria_atual = len(fila) + len(estados_visitados)
        resultado.memoria_usada = max(resultado.memoria_usada, memoria_atual)

        estado_atual, caminho_atual = fila.popleft()
        resultado.estados_explorados += 1

        if len(caminho_atual) >= profundidade_max:
            continue

        for movimento in estado_atual.obter_movimentos_possiveis():
            novo_estado = estado_atual.criar_copia()
            novo_estado.executar_movimento(movimento)

            if novo_estado.solver():
                resultado.tempo_execucao = time.time() - inicio
                resultado.caminho_solucao = caminho_atual + [movimento]
                resultado.sucesso = True
                return resultado

            if novo_estado not in estados_visitados:
                estados_visitados.add(novo_estado)
                fila.append((novo_estado, caminho_atual + [movimento]))

    resultado.tempo_execucao = time.time() - inicio
    return resultado

In [6]:
# IDDFS
def busca_profundidade_limitada_recursiva(cubo, caminho, limite, visitados, stats):
    """Função auxiliar para busca em profundidade limitada"""
    if cubo.solver():
        return caminho

    if len(caminho) >= limite:
        return None

    if cubo in visitados:
        return None

    visitados.add(cubo)
    stats.estados_explorados += 1
    # Atualiza pico de memória
    stats.memoria_usada = max(stats.memoria_usada, len(visitados))

    for movimento in cubo.obter_movimentos_possiveis():
        novo_cubo = cubo.criar_copia()
        novo_cubo.executar_movimento(movimento)

        resultado = busca_profundidade_limitada_recursiva(
            novo_cubo, caminho + [movimento], limite, visitados, stats
        )
        if resultado is not None:
            return resultado

    visitados.remove(cubo)
    return None


def busca_profundidade_iterativa_aprofundamento(cubo_inicial, profundidade_max=100):
    """Implementação da Busca em Profundidade Iterativa"""
    resultado = EstatisticasBusca()
    inicio = time.time()

    for nivel in range(profundidade_max + 1):
        visitados = set()
        caminho = busca_profundidade_limitada_recursiva(
            cubo_inicial, [], nivel, visitados, resultado
        )

        if caminho is not None:
            resultado.tempo_execucao = time.time() - inicio
            resultado.caminho_solucao = caminho
            resultado.sucesso = True
            return resultado

    resultado.tempo_execucao = time.time() - inicio
    return resultado


In [7]:
# A*
def busca_a_estrela_heuristica(cubo_inicial, funcao_heuristica=heuristica_padrao_cruzado, prof_max=1000):
    """Implementação do algoritmo A* com heurística"""
    resultado = EstatisticasBusca()
    inicio = time.time()

    if cubo_inicial.solver():
        resultado.sucesso = True
        resultado.caminho_solucao = []
        resultado.estados_explorados = 0
        resultado.tempo_execucao = time.time() - inicio
        return resultado

    lista_aberta = []
    conjunto_fechado = set()
    contador_nos = 0

    custo_inicial = 0
    heuristica_inicial = funcao_heuristica(cubo_inicial)
    funcao_avaliacao_inicial = custo_inicial + heuristica_inicial

    heappush(lista_aberta, (funcao_avaliacao_inicial, contador_nos, custo_inicial, cubo_inicial, []))
    contador_nos += 1

    while lista_aberta:
        # Calcula pico de memória (lista aberta + conjunto fechado)
        memoria_atual = len(lista_aberta) + len(conjunto_fechado)
        resultado.memoria_usada = max(resultado.memoria_usada, memoria_atual)

        f_atual, _, g_atual, cubo_atual, caminho_atual = heappop(lista_aberta)

        estado_hash = str(cubo_atual.estado.tobytes())
        if estado_hash in conjunto_fechado:
            continue

        conjunto_fechado.add(estado_hash)
        resultado.estados_explorados += 1

        if len(caminho_atual) >= prof_max:
            continue

        for movimento in cubo_atual.obter_movimentos_possiveis():
            novo_cubo = cubo_atual.criar_copia()
            novo_cubo.executar_movimento(movimento)

            if novo_cubo.solver():
                resultado.sucesso = True
                resultado.caminho_solucao = caminho_atual + [movimento]
                resultado.tempo_execucao = time.time() - inicio
                return resultado

            novo_estado_hash = str(novo_cubo.estado.tobytes())
            if novo_estado_hash not in conjunto_fechado:
                novo_g = g_atual + 1
                novo_h = funcao_heuristica(novo_cubo)
                novo_f = novo_g + novo_h

                heappush(lista_aberta, (novo_f, contador_nos, novo_g, novo_cubo, caminho_atual + [movimento]))
                contador_nos += 1

    resultado.tempo_execucao = time.time() - inicio
    return resultado

In [8]:
#Birecional
def movimento_inverso(movimento):
    """Retorna o movimento inverso de um dado movimento"""
    if movimento.endswith("'"):
        return movimento[:-1]
    else:
        return movimento + "'"


def inverter_sequencia(sequencia):
    """Inverte uma sequência de movimentos"""
    return [movimento_inverso(mov) for mov in reversed(sequencia)]


def busca_bidirecional_otimizada(cubo_inicial, prof_max=50):
    """Implementação da Busca Bidirecional"""
    resultado = EstatisticasBusca()
    inicio = time.time()

    if cubo_inicial.solver():
        resultado.tempo_execucao = time.time() - inicio
        resultado.caminho_solucao = []
        resultado.sucesso = True
        return resultado

    fila_progressiva = deque([(cubo_inicial, [])])
    visitados_progressiva = {cubo_inicial: []}

    cubo_resolvido = RubiksCube(cubo_inicial.tamanho)
    fila_regressiva = deque([(cubo_resolvido, [])])
    visitados_regressiva = {cubo_resolvido: []}

    prof_individual = prof_max // 2

    for nivel in range(prof_individual + 1):
        # Expansão progressiva
        proxima_fila_prog = deque()
        while fila_progressiva:
            cubo_atual, caminho = fila_progressiva.popleft()
            resultado.estados_explorados += 1

            if len(caminho) == nivel:
                for movimento in cubo_atual.obter_movimentos_possiveis():
                    novo_cubo = cubo_atual.criar_copia()
                    novo_cubo.executar_movimento(movimento)
                    novo_caminho = caminho + [movimento]

                    if novo_cubo in visitados_regressiva:
                        caminho_regressivo = visitados_regressiva[novo_cubo]
                        movimentos_finais = inverter_sequencia(caminho_regressivo)

                        resultado.caminho_solucao = novo_caminho + movimentos_finais
                        resultado.sucesso = True
                        resultado.tempo_execucao = time.time() - inicio
                        return resultado

                    if novo_cubo not in visitados_progressiva:
                        visitados_progressiva[novo_cubo] = novo_caminho
                        proxima_fila_prog.append((novo_cubo, novo_caminho))

        fila_progressiva = proxima_fila_prog

        # Expansão regressiva
        proxima_fila_regr = deque()
        while fila_regressiva:
            cubo_atual, caminho_aplicado = fila_regressiva.popleft()
            resultado.estados_explorados += 1

            if len(caminho_aplicado) == nivel:
                for movimento in cubo_atual.obter_movimentos_possiveis():
                    novo_cubo = cubo_atual.criar_copia()
                    movimento_rev = movimento_inverso(movimento)
                    novo_cubo.executar_movimento(movimento_rev)

                    novo_caminho_aplicado = caminho_aplicado + [movimento_rev]

                    if novo_cubo in visitados_progressiva:
                        caminho_progressivo = visitados_progressiva[novo_cubo]
                        movimentos_finais = inverter_sequencia(novo_caminho_aplicado)

                        resultado.caminho_solucao = caminho_progressivo + movimentos_finais
                        resultado.sucesso = True
                        resultado.tempo_execucao = time.time() - inicio
                        return resultado

                    if novo_cubo not in visitados_regressiva:
                        visitados_regressiva[novo_cubo] = novo_caminho_aplicado
                        proxima_fila_regr.append((novo_cubo, novo_caminho_aplicado))

        fila_regressiva = proxima_fila_regr

        # Calcula pico de memória (todas as estruturas)
        memoria_atual = (len(fila_progressiva) + len(fila_regressiva) +
                        len(visitados_progressiva) + len(visitados_regressiva))
        resultado.memoria_usada = max(resultado.memoria_usada, memoria_atual)

        if not fila_progressiva and not fila_regressiva:
            break

    resultado.tempo_execucao = time.time() - inicio
    return resultado

In [9]:
# utils

def validacao_matematica(cubo):
    """Validação matemática otimizada - verifica se cada face é uniforme"""
    for face_id in range(6):
        face = cubo.estado[face_id]
        if not np.all(face == face_id):
            return False
    return True


def calcular_memoria_mb(num_objetos, tamanho_cubo=3):
    """Converte número de objetos para MB aproximados"""
    # Tamanho de um cubo: 6 faces × 3×3 × 1 byte = 54 bytes
    tamanho_cubo_bytes = 6 * tamanho_cubo * tamanho_cubo * 1

    # Overhead do Python por objeto (aproximado)
    overhead_python = 200  # bytes por objeto na memória

    # Tamanho total por objeto (cubo + overhead + caminho médio)
    tamanho_por_objeto = tamanho_cubo_bytes + overhead_python + 50  # 50 bytes para caminho

    # Converte para MB
    memoria_bytes = num_objetos * tamanho_por_objeto
    memoria_mb = memoria_bytes / (1024 * 1024)

    return memoria_mb

In [10]:
def salvar_log(conteudo, nome_arquivo="resultado_cubo_rubik.log"):
    """Salva o resultado em arquivo .log"""
    try:
        with open(nome_arquivo, 'w', encoding='utf-8') as arquivo:
            arquivo.write(conteudo)
        print(f"\n Resultados salvos em: {nome_arquivo}")
    except Exception as e:
        print(f"\n Erro ao salvar arquivo: {e}")

In [11]:
def exec():
    """Executa testes dos algoritmos de busca com 5 movimentos"""

    # String para acumular todo o log
    log_completo = ""

    print("🎯 ALGORITMOS DE BUSCA - CUBO DE RUBIK 3x3x3")

    print("=" * 50)

    log_completo += "ALGORITMOS DE BUSCA - CUBO DE RUBIK 3x3x3\n"

    log_completo += "=" * 50 + "\n"

    # Cria cubo base e gera sequência
    cubo_base = RubiksCube(3)
    cubo_temp = cubo_base.criar_copia()
    movimentos = [2,3,4,5]


    # Testa algoritmos
    algoritmos = [
        ("BFS", busca_em_largura),
        ("IDDFS", busca_profundidade_iterativa_aprofundamento),
        ("A*-Cruz", lambda c: busca_a_estrela_heuristica(c, heuristica_padrao_cruzado)),
        ("Bidirecional", busca_bidirecional_otimizada)
    ]
    for mov in movimentos:
        sequencia = cubo_temp.embaralhar_cubo(mov)
        print(f"Sequência: {' '.join(sequencia)}")

        log_completo += f"TESTE COM {mov} MOVIMENTOS\n"
        log_completo += "=" * 50 + "\n"
        print(f"Sequência: {' '.join(sequencia)}")
        print("="*50)
        log_completo += f"Sequência: {' '.join(sequencia)}\n"
        log_completo += "-" * 40 + "\n"
        for nome, algoritmo in algoritmos:
            print(f"\n {nome}-movimentos {mov} :")
            log_completo += f"\n{nome} - movimentos {mov} :\n"

            # Prepara cubo embaralhado
            cubo_embaralhado = cubo_base.criar_copia()
            for movimento in sequencia:
                cubo_embaralhado.executar_movimento(movimento)

            # Executa algoritmo
            resultado = algoritmo(cubo_embaralhado)

            if resultado.sucesso:
                # Converte memória para MB
                memoria_mb = calcular_memoria_mb(resultado.memoria_usada)

                print(f"    Solução: {' '.join(resultado.caminho_solucao)}")
                print(f"    {len(resultado.caminho_solucao)} movimentos")
                print(f"    {resultado.tempo_execucao:.3f}s")
                print(f"    {resultado.estados_explorados} estados explorados")
                print(f"    {memoria_mb:.2f} MB")

                log_completo += f"   Solução: {' '.join(resultado.caminho_solucao)}\n"
                log_completo += f"   Movimentos: {len(resultado.caminho_solucao)}\n"
                log_completo += f"   Tempo: {resultado.tempo_execucao:.3f}s\n"
                log_completo += f"   Estados explorados: {resultado.estados_explorados}\n"
                log_completo += f"   Memória: {memoria_mb:.2f} MB\n"

                # Validação matemática otimizada
                cubo_validacao = cubo_base.criar_copia()

                # Aplica embaralhamento
                for movimento in sequencia:
                    cubo_validacao.executar_movimento(movimento)

                # Aplica solução
                for movimento in resultado.caminho_solucao:
                    cubo_validacao.executar_movimento(movimento)

                # Validação matemática
                if validacao_matematica(cubo_validacao):
                    print(" Validação: CORRETA")
                    log_completo += "   Validação: CORRETA\n"
                    print("="*50)
                else:
                    print("   Validação: FALHOU")
                    log_completo += "   Validação: FALHOU\n"
                    print("="*50)
            else:
                print("   Não encontrou solução")
                log_completo += "   Não encontrou solução\n"
                print("="*50)

    # Salva resultado em arquivo .log
    salvar_log(log_completo)





In [ ]:
exec()

🎯 ALGORITMOS DE BUSCA - CUBO DE RUBIK 3x3x3
Sequência: L B'
Sequência: L B'

 BFS-movimentos 2 :
    Solução: B B B' L'
    4 movimentos
    0.540s
    492 estados explorados
    2.62 MB
 Validação: CORRETA

 IDDFS-movimentos 2 :
    Solução: B B B' L'
    4 movimentos
    0.546s
    636 estados explorados
    0.00 MB
 Validação: CORRETA

 A*-Cruz-movimentos 2 :
    Solução: B B B' L'
    4 movimentos
    0.277s
    164 estados explorados
    0.51 MB
 Validação: CORRETA

 Bidirecional-movimentos 2 :
    Solução: B L'
    2 movimentos
    0.030s
    25 estados explorados
    0.01 MB
   Validação: FALHOU
Sequência: R U' D'
Sequência: R U' D'

 BFS-movimentos 3 :
    Solução: U D R'
    3 movimentos
    0.075s
    42 estados explorados
    0.23 MB
 Validação: CORRETA

 IDDFS-movimentos 3 :
    Solução: U D R'
    3 movimentos
    0.083s
    47 estados explorados
    0.00 MB
 Validação: CORRETA

 A*-Cruz-movimentos 3 :
    Solução: U D R'
    3 movimentos
    0.005s
    3 estados explorado